In [3]:
pip install pesq

  Using cached pesq-0.0.4.tar.gz (38 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for pesq
Failed to build pesq
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [26 lines of output]
  C:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\setuptools\installer.py:27: SetuptoolsDeprecationWarning: setuptools.installer is deprecated. Requirements should be satisfied by a PEP 517 installer.
    warnings.warn(
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-310
  creating build\lib.win-amd64-cpython-310\pesq
  copying pesq\_pesq.py -> build\lib.win-amd64-cpython-310\pesq
  copying pesq\__init__.py -> build\lib.win-amd64-cpython-310\pesq
  copying pesq\cypesq.pyx -> build\lib.win-amd64-cpython-310\pesq
  copying pesq\dsp.h -> build\lib.win-amd64-cpython-310\pesq
  copying pesq\pesq.h -> build\lib.win-amd64-cpython-310\pesq
  copying pesq\pesqio.h -> build\lib.win-amd64-cpython-310\pesq
  copying pesq\pesqmain.h -> build\lib.win-amd64-cpython-310\pesq
  c

In [2]:
pip install torchmetrics[audio]

   ---------------------------------------- 2.3/2.3 MB 3.1 MB/s eta 0:00:00
   ---------------------------------------- 806.1/806.1 kB 3.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: C:\Users\Dell\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [4]:
pip install torchmetrics

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: C:\Users\Dell\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [5]:
pip install torchaudio

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: C:\Users\Dell\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [7]:
import torchaudio
import torch
import torchmetrics
import matplotlib.pyplot as plt
from torch import randn
from torchmetrics.functional.audio.pesq import perceptual_evaluation_speech_quality
from torchaudio.utils import download_asset

# Function to load audio and resample
def load_and_resample_audio(file_path, target_sample_rate):
    waveform, original_sample_rate = torchaudio.load(file_path)
    resampler = torchaudio.transforms.Resample(original_sample_rate, target_sample_rate)
    resampled_waveform = resampler(waveform)
    return resampled_waveform, target_sample_rate

# Function to compute STFT and return magnitude and phase
def compute_stft(audio_waveform, sample_rate, n_fft=400, hop_length=160, win_length=400):
    stft_transform = torchaudio.transforms.Spectrogram(n_fft=n_fft, hop_length=hop_length, win_length=win_length)
    spectrogram = stft_transform(audio_waveform)
    magnitude = torch.abs(spectrogram)
    phase = torch.angle(spectrogram)
    return magnitude, phase

# Function to compute ISTFT and return reconstructed audio waveform
def compute_istft(magnitude, phase, sample_rate, n_fft=400, hop_length=160, win_length=400):
    # Reconstruct complex spectrogram
    spectrogram_complex = magnitude * torch.exp(1j * phase)
    
    # Apply ISTFT using torch.istft
    reconstructed_waveform = torch.istft(
        spectrogram_complex, 
        n_fft=n_fft, 
        hop_length=hop_length, 
        win_length=win_length, 
        window=torch.hann_window(win_length),  # You can use other windows as well
        center=True
    )
    
    return reconstructed_waveform



# Function to compare original and result audio waveforms
def compare_audio(original_waveform, result_waveform):
    assert original_waveform.shape == result_waveform.shape, "Shapes of original and result waveforms must be the same."
    mse_loss = torch.nn.MSELoss()
    mse = mse_loss(original_waveform, result_waveform)
    print(f"Mean Squared Error (MSE): {mse.item()}")

# Function to calculate PESQ and STOI using torchmetrics
def evaluate_audio(original_waveform, result_waveform, sample_rate):
    

# Assuming preds and target are PyTorch tensors
    g = torch.manual_seed(1)
    preds = randn(8000)
    target = randn(8000)

    # Compute PESQ scores
    pesq_nb = perceptual_evaluation_speech_quality(preds, target, 8000, 'nb')
    pesq_wb = perceptual_evaluation_speech_quality(preds, target, 16000, 'wb')

    print(f"PESQ Score (NB): {pesq_nb.item()}")     
    print(f"PESQ Score (WB): {pesq_wb.item()}")

    stoi_metric = torchmetrics.audio.STOI(sample_rate)
    
    stoi_score = stoi_metric(result_waveform, original_waveform)
    print(f"STOI Score: {stoi_score.item()}")

# Sample WAV file path
SAMPLE_WAV = download_asset("tutorial-assets/Lab41-SRI-VOiCES-src-sp0307-ch127535-sg0042.wav")
# Load and resample audio
resampled_waveform, target_sample_rate = load_and_resample_audio(SAMPLE_WAV, 48000)

# Compute STFT
magnitude, phase = compute_stft(resampled_waveform, target_sample_rate)

# Compute ISTFT
reconstructed_waveform = compute_istft(magnitude, phase, target_sample_rate)

# Compare original and result audio
compare_audio(resampled_waveform, reconstructed_waveform)

# Evaluate PESQ and STOI
evaluate_audio(resampled_waveform, reconstructed_waveform, target_sample_rate)

# Plot original and result waveforms
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(resampled_waveform[0].numpy())
plt.title("Original Audio")

plt.subplot(1, 2, 2)
plt.plot(reconstructed_waveform[0].numpy())
plt.title("Reconstructed Audio")

plt.show()


100%|██████████| 106k/106k [00:00<00:00, 290kB/s]  


Mean Squared Error (MSE): 0.6627383232116699


ModuleNotFoundError: PESQ metric requires that pesq is installed. Either install as `pip install torchmetrics[audio]` or `pip install pesq`.

In [8]:
pip install pesq

  Using cached pesq-0.0.4.tar.gz (38 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for pesq
Failed to build pesq
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [26 lines of output]
  C:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\setuptools\installer.py:27: SetuptoolsDeprecationWarning: setuptools.installer is deprecated. Requirements should be satisfied by a PEP 517 installer.
    warnings.warn(
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-310
  creating build\lib.win-amd64-cpython-310\pesq
  copying pesq\_pesq.py -> build\lib.win-amd64-cpython-310\pesq
  copying pesq\__init__.py -> build\lib.win-amd64-cpython-310\pesq
  copying pesq\cypesq.pyx -> build\lib.win-amd64-cpython-310\pesq
  copying pesq\dsp.h -> build\lib.win-amd64-cpython-310\pesq
  copying pesq\pesq.h -> build\lib.win-amd64-cpython-310\pesq
  copying pesq\pesqio.h -> build\lib.win-amd64-cpython-310\pesq
  copying pesq\pesqmain.h -> build\lib.win-amd64-cpython-310\pesq
  c

In [9]:
pip install manimce

     -------------------------------------- 249.7/249.7 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 150.0/150.0 kB 3.0 MB/s eta 0:00:00
   ---------------------------------------- 75.1/75.1 kB 2.1 MB/s eta 0:00:00
   ---------------------------------------- 877.2/877.2 kB 2.5 MB/s eta 0:00:00
  Created wheel for pangocairocffi: filename=pangocairocffi-0.4.0-py3-none-any.whl size=19302 sha256=4526510894847db76f6abd9149d1a7fe46a58146ec972b7d2b8932246c4b469f
  Stored in directory: c:\users\dell\appdata\local\pip\cache\wheels\98\39\39\48f31c814057ce8da1c7d990682b390aecd6b0acf283a9a23b
  Created wheel for pangocffi: filename=pangocffi-0.8.0-py3-none-any.whl size=3791

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastapi 0.104.0 requires typing-extensions>=4.8.0, but you have typing-extensions 3.10.0.2 which is incompatible.
librosa 0.10.1 requires typing-extensions>=4.1.1, but you have typing-extensions 3.10.0.2 which is incompatible.
pydantic 1.10.5 requires typing-extensions>=4.2.0, but you have typing-extensions 3.10.0.2 which is incompatible.
streamlit 1.14.1 requires rich>=10.11.0, but you have rich 6.2.0 which is incompatible.
tensorflow 2.10.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.2 which is incompatible.
tf-nightly-intel 2.13.0.dev20230325 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.
uvicorn 0.23.2 requires typing-extensions>=4.0; python_version < "3.11", but you have typing-extensions 3.10.0.2 wh